In [ ]:
import pandas as pd
import numpy as np

In [ ]:
file_data = pd.read_csv("data.csv",encoding='latin1')
file_data.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,6,7,9,10,11,12,13,15,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Sl. No.,YEAR,MONTH,DATE,ORIGIN TIME,Unnamed: 5,MAGNITUDE,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,LAT (N),LONG (E),DEPTH (km),INTENSITY,Unnamed: 16,Unnamed: 17,LOCATION,REFERENCE
0,NaN,NaN,NaN,NaN,(UTC),(IST),Mw,Mw,Mb,Mb,Ms,ML,NaN,NaN,NaN,MM,MMI,MME,NaN,NaN
1,1.0,-2474.0,0,0.0,NaN,NaN,7.5,7.5,NaN,6.969202899,7.260619977,7.427072403,71,24,0.0,NaN,NaN,NaN,NaN,Dr STGR
2,2.0,-325.0,0,0.0,NaN,NaN,7.5,7.5,NaN,6.969202899,7.260619977,7.427072403,71,24,0.0,NaN,NaN,NaN,NaN,Dr STGR
3,3.0,25.0,0,0.0,NaN,NaN,7.5,7.5,NaN,6.969202899,7.260619977,7.427072403,72.9,33.72,0.0,NaN,NaN,NaN,NaN,Dr STGR
4,4.0,26.0,5,10.0,08.19.10.0,NaN,6.1397,6.1397,5.737047101,5.737047101,5.698851894,5.999685205,17.3,80.1,NaN,NaN,NaN,NaN,NaN,NEIC


In [ ]:
# drop row 0
file_data = file_data.drop(file_data.index[0])
file_data.head()

,Sl. No.,YEAR,MONTH,DATE,ORIGIN TIME,Unnamed: 5,MAGNITUDE,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,LAT (N),LONG (E),DEPTH (km),INTENSITY,Unnamed: 16,Unnamed: 17,LOCATION,REFERENCE
1,1.0,-2474.0,0,0.0,NaN,NaN,7.5,7.5,NaN,6.969202899,7.260619977,7.427072403,71,24,0.0,NaN,NaN,NaN,NaN,Dr STGR
2,2.0,-325.0,0,0.0,NaN,NaN,7.5,7.5,NaN,6.969202899,7.260619977,7.427072403,71,24,0.0,NaN,NaN,NaN,NaN,Dr STGR
3,3.0,25.0,0,0.0,NaN,NaN,7.5,7.5,NaN,6.969202899,7.260619977,7.427072403,72.9,33.72,0.0,NaN,NaN,NaN,NaN,Dr STGR
4,4.0,26.0,5,10.0,08.19.10.0,NaN,6.1397,6.1397,5.737047101,5.737047101,5.698851894,5.999685205,17.3,80.1,NaN,NaN,NaN,NaN,NaN,NEIC
5,5.0,26.0,5,10.0,08.19.10.0,NaN,6.1397,6.1397,5.737047101,5.737047101,6.075520196,5.999685205,26,97,80.0,NaN,NaN,NaN,Myanmar,G-R


In [ ]:
# shuffle rows
# file_data = file_data.sample(frac = 1) 
# file_data.tail()

In [ ]:
import re
def edit_latitude(val):
  if type(val) == str:
    new_val = val.replace('?','').replace(' ','').replace(',','.')
    new_val = re.sub('[^a-zA-Z0-9\.]', '', new_val)
    if new_val[-1] == 'N':
        new_val = float(new_val[0:-1])
    elif new_val[-1] == 'S':
        new_val = 180 -1*float(new_val[0:-1])
    elif new_val[-1] == 'E':
        new_val = float(new_val[0:-1])    
    else:
      new_val = float(new_val)  
    return new_val
  return float(val)

In [ ]:
import re
def edit_longitude(val):
  if type(val) == str:
    new_val = val.replace('?','').replace(' ','').replace(',','.')
    new_val = re.sub('[^a-zA-Z0-9\.]', '', new_val)
    if new_val[-1] == 'E':
        new_val = float(new_val[0:-1])
    elif new_val[-1] == 'W':
        new_val = 360 - 1*float(new_val[0:-1])
    elif new_val[-1] == 'S':
        new_val = float(new_val[0:-1])
    else:
      new_val = float(new_val)  
    return new_val
  return float(val)

In [ ]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [ ]:
# file_data.columns
# filter data
req_features = ['LAT (N)','LONG (E)','DEPTH (km)','MAGNITUDE']
filtered_data = file_data[req_features]
print(filtered_data.shape)
filtered_data['LAT (N)'] = filtered_data['LAT (N)'].apply(edit_latitude)
filtered_data['LONG (E)'] = filtered_data['LONG (E)'].apply(edit_longitude)
filtered_data = clean_dataset(filtered_data)
print(filtered_data.shape)

(52989, 4)
(40107, 4)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

In [ ]:
from sklearn import preprocessing
lab_enc = preprocessing.LabelEncoder()

x_features = ['LAT (N)','LONG (E)','DEPTH (km)']
X = filtered_data[x_features]
y = filtered_data['MAGNITUDE']
# X = lab_enc.fit_transform(X)
# z = lab_enc.fit_transform(y)
print(X.shape)
print(y.shape)

(40107, 3)
(40107,)


In [ ]:
threshold = 4.4
y = y.astype('float')
y_m = y.copy()
y_m[y_m < threshold] = 0
y_m[y_m >= threshold] = 1
y = y_m.copy()
print(y)

1        1.0
2        1.0
3        1.0
5        1.0
6        1.0
        ... 
52985    0.0
52986    0.0
52987    0.0
52988    0.0
52989    0.0
Name: MAGNITUDE, Length: 40107, dtype: float64


In [ ]:
# convert y from object to float
# print(y.dtype)
# def convertToFloat(val):
  # return float(val)
# y = y.apply(convertToFloat)
# print(y.dtype)
# y.tail()

In [ ]:
# test-train split
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3,random_state=1)
train_X.describe()
print(np.any(np.isnan(train_y)))

False


In [ ]:
# reshape y
# y = np.reshape(y,(-1, 1))

# from sklearn.impute import SimpleImputer
# my_imputer = SimpleImputer()
# imputed_train_X = pd.DataFrame(my_imputer.fit_transform(train_X))
# imputed_test_X = pd.DataFrame(my_imputer.transform(test_X))

# imputed_train_y = pd.DataFrame(my_imputer.fit_transform(train_y))
# imputed_test_y = pd.DataFrame(my_imputer.transform(test_y))


# imputed_train_X.columns = train_X.columns
# imputed_test_X.columns = test_X.columns
# imputed_train_y.columns = train_y.columns
# imputed_test_y.columns = test_y.columns

In [ ]:
# KNN
from sklearn.neighbors import KNeighborsClassifier as KNNmodel
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_absolute_error
def getKNN(neighbours):
  knn = KNNmodel(n_neighbors=neighbours)
  knn.fit(train_X, train_y)
  y_pred = knn.predict(test_X)
  # print(classification_report(test_y, y_pred))
  # roc_auc_score(test_y, y_pred)
  mae = mean_absolute_error(y_pred, test_y)
  return mae

In [ ]:
# for i in range(88,110):
print(getKNN(49))

0.3807861713620876


In [ ]:
# Decision Tree
from sklearn.tree import DecisionTreeRegressor
def get_mae(max_leaf_nodes, train_X, test_X, train_y, test_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=1)
    # print(train_X.shape)
    model.fit(train_X, train_y)
    preds_val = model.predict(test_X)
    mae = mean_absolute_error(test_y, preds_val)
    from sklearn.metrics import roc_auc_score
    print(roc_auc_score(test_y, preds_val))
    return(mae)

In [ ]:
print(get_mae(10000, train_X, test_X, train_y, test_y))

0.579917493543943
0.41557522369040695
